In [2]:
# 1. Import libraries
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler

# 2. Load dataset
df = pd.read_csv("Dataset .csv")

# 3. Keep a display copy before dropping columns
df_display = df[['Restaurant Name', 'Cuisines', 'City', 'Average Cost for two', 'Aggregate rating']].copy()

# 4. Drop irrelevant columns for processing
df.drop(columns=[
    'Restaurant ID', 'Restaurant Name', 'Address', 'Locality',
    'Locality Verbose', 'Rating color', 'Rating text'
], inplace=True)

# 5. Fix the FutureWarning
df['Cuisines'] = df['Cuisines'].fillna(df['Cuisines'].mode()[0])

# 6. Filter necessary columns
df_reco = df[['Cuisines', 'Average Cost for two', 'Price range', 'Aggregate rating', 'City']]

# 7. One-hot encode
df_encoded = pd.get_dummies(df_reco, columns=['Cuisines', 'City'], drop_first=True)

# 8. Scale numeric columns
scaler = StandardScaler()
df_encoded[['Average Cost for two', 'Price range', 'Aggregate rating']] = scaler.fit_transform(
    df_encoded[['Average Cost for two', 'Price range', 'Aggregate rating']]
)

# 9. Recommendation function
def recommend_restaurants(preferred_cuisine, preferred_city, max_cost, min_rating, top_n=5):
    # Filter restaurants based on user input
    filtered = df[
        (df['Cuisines'].str.contains(preferred_cuisine, case=False, na=False)) &
        (df['City'].str.contains(preferred_city, case=False, na=False)) &
        (df['Average Cost for two'] <= max_cost) &
        (df['Aggregate rating'] >= min_rating)
    ]

    if filtered.empty:
        return "❌ No matching restaurants found."

    # Build a user profile from the filtered set
    user_profile = df_encoded.loc[filtered.index].mean().values.reshape(1, -1)

    # Compute similarity
    similarity_scores = cosine_similarity(user_profile, df_encoded)[0]

    # Get top recommendations
    top_indices = similarity_scores.argsort()[-top_n:][::-1]
    recommendations = df_display.iloc[top_indices]

    return recommendations

# 10. Test the recommender
recommend_restaurants(
    preferred_cuisine="Italian",
    preferred_city="Delhi",
    max_cost=1500,
    min_rating=3.5,
    top_n=5
)


,Restaurant Name,Cuisines,City,Average Cost for two,Aggregate rating
3258,Cafe Yell,"Cafe, Continental, Italian",New Delhi,1000,4.4
7196,Puppychino,"Cafe, Continental, Italian",New Delhi,1000,4.4
4639,Big Chill,"Italian, Continental, European, Cafe",New Delhi,1500,4.6
4638,Big Chill,"Italian, Continental, European, Cafe",New Delhi,1500,4.5
3012,MOB Brewpub,"Continental, Italian, Asian, Indian",New Delhi,1500,4.7
